In [61]:
import argparse

import h5py as h5
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Creme-ML
from creme import optim
from creme.linear_model import LogisticRegression
from creme.multiclass import OneVsRestClassifier
from creme.preprocessing import StandardScaler
from creme.compose import Pipeline
from creme import metrics
from creme import stream
from creme import compat
from creme import model_selection

from h5imagegenerator import HDF5ImageGenerator

In [62]:
np.random.seed(42)

### Logistic Regression

In [63]:
# Features generator
gen = HDF5ImageGenerator(
    src= 'features_train.h5',
    classes_key='classes',
    num_classes=2,
    labels_encoding=False,
    scaler=False,
    batch_size=16
)

In [64]:
model = Pipeline()
model |= LogisticRegression()

model

Pipeline (
  LogisticRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Log ()
    l2=0.
    intercept=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

In [59]:
param_grid = model_selection.expand_param_grid({
    'LogisticRegression': {
        'optimizer': [
            (optim.SGD, {'lr': [0.1, 0.01, 0.005]}),
        ]
    }
})

In [72]:
models = [model._set_params(params) for params in param_grid]

sh = model_selection.SuccessiveHalvingClassifier(
    models=models,
    metric=metrics.Accuracy(),
    budget=204800,
    eta=2,
    verbose=True,
)

ValueError: math domain error

In [43]:
metric = Accuracy()

In [44]:
cm = ConfusionMatrix()

In [45]:
columns = [f'feature_{i}' for i in range(204800)]

In [46]:
for batch_X, batch_y in gen:
    for X, y in zip(batch_X, batch_y):
        X_as_dict = dict(zip(columns, X))

        y_hat = model.predict_one(X_as_dict)
        model = model.fit_one(X_as_dict, y)
        
        # Update the metrics
        metric = metric.update(y_true=y, y_pred=y_hat)
        cm = cm.update(y_true=y, y_pred=y_hat)
    
    print(f'Accuracy at batch is {metric}.')

Accuracy at batch is Accuracy: 56.25%.
Accuracy at batch is Accuracy: 68.75%.
Accuracy at batch is Accuracy: 75.00%.
Accuracy at batch is Accuracy: 78.13%.
Accuracy at batch is Accuracy: 78.75%.
Accuracy at batch is Accuracy: 80.21%.
Accuracy at batch is Accuracy: 81.25%.
Accuracy at batch is Accuracy: 78.91%.
Accuracy at batch is Accuracy: 79.17%.
Accuracy at batch is Accuracy: 80.62%.
Accuracy at batch is Accuracy: 81.82%.
Accuracy at batch is Accuracy: 81.77%.
Accuracy at batch is Accuracy: 82.21%.
Accuracy at batch is Accuracy: 83.48%.
Accuracy at batch is Accuracy: 82.92%.
Accuracy at batch is Accuracy: 83.59%.
Accuracy at batch is Accuracy: 83.82%.
Accuracy at batch is Accuracy: 84.03%.
Accuracy at batch is Accuracy: 84.54%.
Accuracy at batch is Accuracy: 84.69%.
Accuracy at batch is Accuracy: 84.82%.
Accuracy at batch is Accuracy: 84.94%.
Accuracy at batch is Accuracy: 85.60%.
Accuracy at batch is Accuracy: 85.68%.


KeyboardInterrupt: 

### Cross-Validation Score

In [ ]:
metric

In [ ]:
cm